#  preprocessing

In [69]:
import os
import shutil
import sys

import numpy as np
from scipy import sparse
import pandas as pd
import math
import matplotlib.pyplot as plt
import random

### Load Data

In [70]:
### change `DATA_DIR` to the location where movielens-20m dataset sits
DATA_DIR = 'C:/Users/SEO/PycharmProjects/testdigo/datasets/ML_20m'

In [71]:
pro_dir = './'
cur_min_uc = 5
cur_val_num = 10000
cur_te_num =10000
cur_n_items_u = 10
cur_train_por = 7
cur_val_por = 3
cur_test_por =3

In [72]:
raw_data_total = pd.read_csv(os.path.join(DATA_DIR, 'ratings.csv'), header=0)
print(raw_data_total.shape[0])
raw_data_total = raw_data_total[raw_data_total['rating'] > 3.5]
raw_data = pd.DataFrame({'count' : raw_data_total.groupby( [ "userId", "movieId"] ).size()}).reset_index()
del raw_data['count']
print(raw_data)
print(raw_data.shape)

20000263
         userId  movieId
0             1      151
1             1      223
2             1      253
3             1      260
4             1      293
...         ...      ...
9995405  138493    66762
9995406  138493    68319
9995407  138493    68954
9995408  138493    69526
9995409  138493    70286

[9995410 rows x 2 columns]
(9995410, 2)


In [73]:
a = raw_data_total.groupby("userId").size()
print(a.shape)

(138287,)


In [74]:
b = raw_data_total.groupby("movieId").size()
print(b.shape)

(20720,)


In [62]:
b = raw_data.groupby("userId").size()
print(b.sort_values(ascending= False))
b[1137]

userId
125794    3177
82418     2655
8405      2503
118205    2377
131894    2349
          ... 
93335        1
27120        1
32451        1
53274        1
66596        1
Length: 138287, dtype: int64


17

In [63]:
raw_data.head()

,userId,movieId
0,1,151
1,1,223
2,1,253
3,1,260
4,1,293


In [64]:
raw_data.shape

(9995410, 2)

### Data splitting procedure

get_count id 별로 그룹화, 그룹별 데이터 수 확인

In [65]:
def get_count(tp, id):
    playcount_groupbyid = tp[[id]].groupby(id, as_index=False)
    count = playcount_groupbyid.size()
    return count

In [66]:
def filter_triplets(tp, min_uc=5, min_sc=0):
    # Only keep the triplets for items which were clicked on by at least min_sc users. 
    if min_sc > 0:
         #기존 data를 min_sc 해당되는 movield만 filtering       
        itemcount = get_count(tp, 'movieId')
        tp = tp[tp['movieId'].isin(itemcount.index[itemcount >= min_sc])]
    
    # Only keep the triplets for users who clicked on at least min_uc items
    # After doing this, some of the items will have less than min_uc users, but should only be a small proportion
    if min_uc > 0:
        usercount = get_count(tp, 'userId')
        #기존 data를 min_uc이상 해당되는 userld만 filtering
        tp = tp[tp['userId'].isin(usercount.index[usercount >= min_uc])]
    
    # Update both usercount and itemcount after filtering
    usercount, itemcount = get_count(tp, 'userId'), get_count(tp, 'movieId') 
    return tp, usercount, itemcount

In [67]:
raw_data, user_activity, item_popularity = filter_triplets(raw_data, 5, 10)

sparsity 구하기 raw_data의 열의 갯수(filtering 된 raw data 총 갯수) / userid 갯수 * products 갯수

In [68]:
sparsity = 1. * raw_data.shape[0] / (user_activity.shape[0] * item_popularity.shape[0])

print("After filtering, there are %d watching events from %d users and %d products (sparsity: %.3f%%)" % 
      (raw_data.shape[0], user_activity.shape[0], item_popularity.shape[0], sparsity * 100))

After filtering, there are 9962997 watching events from 136669 users and 11518 products (sparsity: 0.633%)


In [14]:
print(get_count(raw_data,'userId').sort_values(ascending= False))

userId
125794    3177
82418     2655
8405      2503
118205    2377
131894    2349
          ... 
36078        5
111773       5
123722       5
44769        5
17566        5
Length: 136677, dtype: int64


In [15]:
def getportion(val, test):
    total = user_activity.index.shape[0]
    return round(((total-val-test)/total)*100), round((val/total)*100), round((test/total)*100)

In [16]:
def getDataindex(val,test):
    trainP,valP,testP = getportion(val,test)
    print(trainP, valP, testP)
    totalP = trainP + valP+ testP
    totalUser = user_activity.sort_values(ascending= False)
    sortedIndexUser = totalUser.reset_index()
    print(sortedIndexUser)
    #sortedIndexUser = sortedIndexUser[10:]
    print(sortedIndexUser)
    sortedIndex = sortedIndexUser['userId']
    sortedIndex_tr = sortedIndex[(sortedIndex.index%totalP>=0) & (sortedIndex.index%totalP<trainP)]
    sortedIndex_va = sortedIndex[(sortedIndex.index%totalP>=trainP) & (sortedIndex.index%totalP<(trainP+valP))]
    sortedIndex_te = sortedIndex[(sortedIndex.index%totalP>=(trainP+valP)) & (sortedIndex.index%totalP<(trainP+valP+testP))]
    sorteduid = pd.concat([sortedIndex_tr,sortedIndex_va,sortedIndex_te])
    return sorteduid.tolist(), sortedIndex_tr.tolist(),sortedIndex_va.tolist(), sortedIndex_te.tolist()

In [17]:
def getDataindex_p(trainP,valP,testP):
    print(trainP, valP, testP)
    totalP = trainP + valP+ testP
    totalUser = user_activity.sort_values(ascending= False)
    sortedIndexUser = totalUser.reset_index()
    print(sortedIndexUser)
    #sortedIndexUser = sortedIndexUser[10:]
    print(sortedIndexUser)
    sortedIndex = sortedIndexUser['userId']
    sortedIndex_tr = sortedIndex[(sortedIndex.index%totalP>=0) & (sortedIndex.index%totalP<trainP)]
    sortedIndex_va = sortedIndex[(sortedIndex.index%totalP>=trainP) & (sortedIndex.index%totalP<(trainP+valP))]
    sortedIndex_te = sortedIndex[(sortedIndex.index%totalP>=(trainP+valP)) & (sortedIndex.index%totalP<(trainP+valP+testP))]
    sorteduid = pd.concat([sortedIndex_tr,sortedIndex_va,sortedIndex_te])
    return sorteduid.tolist(), sortedIndex_tr.tolist(),sortedIndex_va.tolist(), sortedIndex_te.tolist()

In [18]:
def getDataindex_order_p(trainP,valP,testP):
    print(trainP, valP, testP)
    totalP = trainP + valP+ testP
    totalUser = user_activity.sort_values(ascending= False)
    sortedIndexUser = totalUser.reset_index()
    #sortedIndexUser = sortedIndexUser[10:]
    print(sortedIndexUser)
    sortedIndex = sortedIndexUser['userId']
    tr_index = round(sortedIndexUser.shape[0]*(trainP/totalP))
    vr_index = round(tr_index +sortedIndexUser.shape[0]*(valP/totalP))
    print("trindex ", tr_index)
    print("vindex",vr_index)
    sortedIndex_tr = sortedIndex[:tr_index]
    sortedIndex_va = sortedIndex[tr_index:vr_index]
    sortedIndex_te = sortedIndex[vr_index:]
    sorteduid = pd.concat([sortedIndex_tr,sortedIndex_va,sortedIndex_te])
    return sorteduid.tolist(), sortedIndex_tr.tolist(),sortedIndex_va.tolist(), sortedIndex_te.tolist()

In [19]:
def RandomgetDataindex(val,test):
    trainP,valP,testP = getportion(val,test)
    print(trainP, valP, testP)
    totalP = trainP + valP+ testP
    totalUser = user_activity.sort_values(ascending= False)
    sortedIndexUser = totalUser.reset_index()
    #sortedIndexUser = sortedIndexUser[10:]
    sortedIndex = sortedIndexUser['userId'].tolist()
    train = list()
    val = list()
    test = list()
    curarr = list()
    unique = list()
    for i in range(len(sortedIndex)):
        curarr.append(sortedIndex[i])
        if len(curarr) == totalP:
            curval = random.sample(curarr,valP)
            val += curval
            for k in curval :
                curarr.remove(k)
            testval = random.sample(curarr,testP)
            test += testval
            for k in testval :
                curarr.remove(k)
            train += curarr
            curarr.clear()
    if len(curarr) !=0 :
        train += curarr
    unique = train + val + test
    print(set(sortedIndex)-set(unique))
    print(set(train).intersection(val))
    print(set(train).intersection(test))
    print(set(val).intersection(test))
    return unique,train,val,test

In [20]:
unique_uid, tr_users, vd_users, te_users = RandomgetDataindex(cur_val_num,cur_te_num)
print(len(unique_uid))
print(len(tr_users))
print(len(te_users))
print(len(te_users))
print(tr_users[0])
print(vd_users[0])
print(te_users[0])

85 7 7
set()
set()
set()
set()
136677
117357
9660
9660
125794
133811
87561


unique_uid, tr_users, vd_users, te_users = getDataindex_order_p(cur_train_por,cur_val_por,cur_test_por)
print(unique_uid[0])
print(tr_users[0])
print(tr_users[-1])
print(vd_users[0])
print(vd_users[-1])
print(te_users[0])
print(te_users[-1])
print(unique_uid[-1])

unique_uid, tr_users, vd_users, te_users = getDataindex(cur_val_num,cur_te_num)
print(len(unique_uid))
print(len(tr_users))
print(len(vd_users))
print(len(te_users))

In [21]:
train_plays = raw_data.loc[raw_data['userId'].isin(tr_users)]
print(pd.unique(train_plays['userId']).shape)
print(train_plays.shape)

(117357,)
(8577238, 2)


In [22]:
unique_sid = pd.unique(train_plays['movieId'])
print(unique_sid.shape)

(20232,)


In [23]:
print(len(unique_uid))
vad_plays = raw_data.loc[raw_data['userId'].isin(vd_users)]
vad_before_set = set(pd.unique(vad_plays['userId']))
vad_plays = vad_plays.loc[vad_plays['movieId'].isin(unique_sid)]
vad_set = set(pd.unique(vad_plays['userId']))
remove_vad_uidx = vad_before_set - vad_set
print(remove_vad_uidx)
print(len(vad_before_set))
print(len(vad_set))
for x in remove_vad_uidx:
    if x in unique_uid:
                unique_uid.remove(x)
test_plays = raw_data.loc[raw_data['userId'].isin(te_users)]
test_before_set = set(pd.unique(test_plays['userId']))
test_plays = test_plays.loc[test_plays['movieId'].isin(unique_sid)]
test_set = set(pd.unique(test_plays['userId']))
remove_test_uidx = test_before_set-test_set
print(len(unique_uid))
print(remove_test_uidx)
for x in remove_test_uidx:
    if x in unique_uid:
                unique_uid.remove(x)
print(len(test_before_set))
print(len(test_set))
print(len(unique_uid))
before_uid = len(unique_uid)
before_vd = pd.unique(vad_plays['userId']).shape
before_te = pd.unique(test_plays['userId']).shape

136677
set()
9660
9660
136677
set()
9660
9660
136677


In [24]:
print(train_plays)

         userId  movieId
0             1      151
1             1      223
2             1      253
3             1      260
4             1      293
...         ...      ...
9995405  138493    66762
9995406  138493    68319
9995407  138493    68954
9995408  138493    69526
9995409  138493    70286

[8577238 rows x 2 columns]


In [25]:
print(vad_plays)

         userId  movieId
88            2        3
89            2       62
90            2       70
91            2      110
92            2      260
...         ...      ...
9993405  138468    88129
9993406  138468    94466
9993407  138468   109487
9993408  138468   111759
9993409  138468   112556

[707045 rows x 2 columns]


In [26]:
print(test_plays)

         userId  movieId
131           3        1
132           3       32
133           3       50
134           3      175
135           3      223
...         ...      ...
9992905  138460     2571
9992906  138460     4226
9992907  138460     4973
9992908  138460     4993
9992909  138460     7153

[705864 rows x 2 columns]


In [27]:
def getSparsity(data):
    usercount = get_count(data,'userId')
    itemcount = get_count(data, 'movieId')
    sparsity = 1. * data.shape[0] / (usercount.shape[0] * len(unique_sid))
    print("After filtering, there are %d watching events from %d users and %d products (sparsity: %.3f%%)" % 
      (data.shape[0], usercount.shape[0], len(unique_sid), sparsity * 100))   

In [28]:
getSparsity(train_plays)

After filtering, there are 8577238 watching events from 117357 users and 20232 products (sparsity: 0.361%)


In [29]:
getSparsity(vad_plays)

After filtering, there are 707045 watching events from 9660 users and 20232 products (sparsity: 0.362%)


In [30]:
getSparsity(test_plays)

After filtering, there are 705864 watching events from 9660 users and 20232 products (sparsity: 0.361%)


In [31]:
def split_train_test_proportion(data,cur_num_items,test_prop=0.2):
    data_grouped_by_user = data.groupby('userId')
    tr_list, te_list = list(), list()
    np.random.seed(12)

    for i, (index, group) in enumerate(data_grouped_by_user):
        n_items_u = len(group)
        if n_items_u >= cur_num_items:
            idx = np.zeros(n_items_u, dtype='bool')
            idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True

            tr_list.append(group[np.logical_not(idx)])
            te_list.append(group[idx])
        else :
            if index in unique_uid:
                unique_uid.remove(index)
        if i % 1000 == 0:
            print("%d users sampled" % i)
            sys.stdout.flush()
    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)
    return data_tr, data_te

In [32]:
vad_plays_tr, vad_plays_te = split_train_test_proportion(vad_plays,cur_n_items_u)

0 users sampled
1000 users sampled
2000 users sampled
3000 users sampled
4000 users sampled
5000 users sampled
6000 users sampled
7000 users sampled
8000 users sampled
9000 users sampled


In [33]:
vad_palys_list = vad_plays_tr['userId']

In [34]:
vad_palys_list_te = vad_plays_te['userId']

In [35]:
print(len(tr_users))

117357


In [36]:
print(pd.unique(vad_plays_tr['userId']).shape)
print(pd.unique(vad_plays_te['userId']).shape)

(9177,)
(9177,)


In [37]:
test_plays_tr, test_plays_te = split_train_test_proportion(test_plays,cur_n_items_u)

0 users sampled
1000 users sampled
2000 users sampled
3000 users sampled
4000 users sampled
5000 users sampled
6000 users sampled
7000 users sampled
8000 users sampled
9000 users sampled


In [38]:
print(pd.unique(test_plays_tr['userId']).shape)

(9178,)


In [39]:
print(pd.unique(test_plays_te['userId']).shape)

(9178,)


In [40]:
print(len(unique_uid))

135712


In [41]:
print(before_uid - len(unique_uid))
print(before_te[0] - pd.unique(test_plays_tr['userId']).shape[0])
print(before_vd[0]- pd.unique(vad_plays_tr['userId']).shape[0])

965
482
483


    unique_sid는 train_plays에서 뽑는 unique한 users

    unique_uid는 뒤섞인것 기준
    show2id는 index와 value를 바꾼 dict (unique_sid에서 뽑은 것 mapping) -> products : index
    profile2id는 index와 value를 바꾼 dict (전체 unique_uid에서 뽑은 것 mapping) -> users : index

In [42]:
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

    unique_sid는 unique_sid.txt로 기록 

In [43]:
if not os.path.exists(pro_dir):
    os.makedirs(pro_dir)

with open(os.path.join(pro_dir, 'unique_sid_2.txt'), 'w') as f:
    for sid in unique_sid:
        f.write('%s\n' % sid)

    split_train_test_proportion함수 data를 userId로 그룹화
    tr_list, te_list은 userId로 그룹화 된 data를 test_prop 만큼 비율로 나누어서 저장 
    te_list는 그룹화된 (int) data 갯수 * test_prop 개의 random한 index를 가지는 data 저장
    tr_list는 나머지 그룹화된 data 저장

### Save the data into (user_index, item_index) format

    numerize 함수 userId와 movieId를 index로 mapping 후 uid(users) sid(products)로 dataFrame생성

In [44]:
def numerize(tp):
    uid = list(map(lambda x: profile2id[x], tp['userId']))
    sid = list(map(lambda x: show2id[x], tp['movieId']))
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

In [45]:
train_data = numerize(train_plays)
train_data.to_csv(os.path.join(pro_dir, 'train_2.csv'), index=False)

In [46]:
def getSparsity_S(data):
    usercount = get_count(data,'uid')
    itemcount = get_count(data, 'sid')
    sparsity = 1. * data.shape[0] / (usercount.shape[0] * len(unique_sid))
    print("After filtering, there are %d watching events from %d users and %d products (sparsity: %.3f%%)" % 
      (data.shape[0], usercount.shape[0], len(unique_sid), sparsity * 100))   

In [47]:
getSparsity_S(train_data)

After filtering, there are 8579649 watching events from 117357 users and 20233 products (sparsity: 0.361%)


In [48]:
vad_data_tr = numerize(vad_plays_tr)
vad_data_tr.to_csv(os.path.join(pro_dir, 'validation_tr_2.csv'), index=False)

In [49]:
getSparsity_S(vad_data_tr)

After filtering, there are 564564 watching events from 9177 users and 20233 products (sparsity: 0.304%)


In [50]:
vad_data_te = numerize(vad_plays_te)
vad_data_te.to_csv(os.path.join(pro_dir, 'validation_te_2.csv'), index=False)

In [51]:
getSparsity_S(vad_data_te)

After filtering, there are 136612 watching events from 9177 users and 20233 products (sparsity: 0.074%)


In [52]:
test_data_tr = numerize(test_plays_tr)
test_data_tr.to_csv(os.path.join(pro_dir, 'test_tr_2.csv'), index=False)

In [53]:
getSparsity_S(test_data_tr)

After filtering, there are 565354 watching events from 9178 users and 20233 products (sparsity: 0.304%)


In [54]:
test_data_te = numerize(test_plays_te)
test_data_te.to_csv(os.path.join(pro_dir, 'test_te_2.csv'), index=False)

In [55]:
getSparsity_S(test_data_te)

After filtering, there are 136831 watching events from 9178 users and 20233 products (sparsity: 0.074%)
